# Individual netCDF Files for Each Variable (mean, clim, anom,std)

In [1]:
import os
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib
import datetime
from dateutil.relativedelta import relativedelta

In [2]:
os.chdir('d:/CMAP')
ds_precip = xr.open_dataset('precip.mon.mean.nc') # ds_precip

# cut new clim

- precip(monthly mean)
- clim_ds
- anom_ds
- stdd_ds

In [3]:
# new_clim_ds
new_clim = ds_precip.sel(time=slice('1991-01-01','2020-12-01'))
new_clim

<xarray.Dataset>
Dimensions:  (time: 360, lat: 72, lon: 144)
Coordinates:
  * time     (time) datetime64[ns] 1991-01-01 1991-02-01 ... 2020-12-01
  * lon      (lon) float32 1.25 3.75 6.25 8.75 11.25 ... 351.2 353.8 356.2 358.8
  * lat      (lat) float32 88.75 86.25 83.75 81.25 ... -83.75 -86.25 -88.75
Data variables:
    precip   (time, lat, lon) float32 ...

In [4]:
# Unit : mm/day -> mm/month
for i in range(len(new_clim.precip)):
    p_time = str(new_clim.time[i].values)[:10] # '1979-03-01'
    b = datetime.datetime.strptime(p_time,'%Y-%m-%d')
    nums_of_day = ((b + relativedelta(months=i))-datetime.timedelta(days=1)).day
    new_clim.precip[i] *= nums_of_day
precip = new_clim.precip

lon = new_clim.lon
lat = new_clim.lat
time = new_clim.time

### ◆ precip nc file (monthly mean)

In [5]:
## monthly ds
precip_arr = np.array(precip)

precip_ds = xr.Dataset(
    data_vars= dict(
        precip = (['time','lat','lon'], precip_arr)
    ),
    coords=dict(
        lon = lon,
        lat = lat,
        time = time
    )
)

## apply attributes
precip_ds.lon.attrs = {'units':'degree_east','long_name':'Longitude'}
precip_ds.lat.attrs = {'units':'degree_north','long_name': 'Latitude'}
precip_ds.precip.attrs ={'units':'mm/month','long_name':'Average Monthly Rate of Precipitation'}


In [6]:
precip_ds

<xarray.Dataset>
Dimensions:  (time: 360, lat: 72, lon: 144)
Coordinates:
  * lon      (lon) float32 1.25 3.75 6.25 8.75 11.25 ... 351.2 353.8 356.2 358.8
  * lat      (lat) float32 88.75 86.25 83.75 81.25 ... -83.75 -86.25 -88.75
  * time     (time) datetime64[ns] 1991-01-01 1991-02-01 ... 2020-12-01
Data variables:
    precip   (time, lat, lon) float32 15.5 15.5 15.5 15.81 ... 0.0 0.0 0.0 0.0

In [7]:
# precip_ds.to_netcdf('8. CMAP_clim_anom_ncfile/individual_ncfile/CMAP_precip_mon_mean_new.nc')

### ◆ clim nc file

In [8]:
clim_sel = new_clim['precip'].groupby('time.month').mean('time')
# new_clim['new_clim'] = clim_sel

## Change the moth dim to the time dim
clim_extend_lst = []

for i in range(30):
    for j in clim_sel.values:
            clim_extend_lst.append(j)

clim_extend_arr = np.array(clim_extend_lst)
        
clim_ds = xr.Dataset(
    data_vars=dict(
        new_clim = (['time','lat','lon'],clim_extend_arr)
    ),
    coords= dict(
        lon = lon,
        lat = lat,
        time = time
    )
)       

In [9]:
## apply attributes
clim_ds.lon.attrs = {'units':'degree_east','long_name':'Longitude'}
clim_ds.lat.attrs = {'units':'degree_north','long_name': 'Latitude'}
clim_ds.new_clim.attrs ={'units':'mm/month','long_name':'Average Long Term Mean Monthly Rate of Precipitation'}

In [10]:
# os.getcwd()
# clim_ds.to_netcdf('8. CMAP_clim_anom_ncfile/individual_ncfile/CMAP_clim_new.nc')

### ◆ anom nc file

In [11]:
anom_lst = []
for  q in range(1, 13):
    mon_group = new_clim.sel(time=new_clim['time.month'] == q)  # month
    anom_sel = mon_group['precip'] - clim_sel.sel(month=q)  # anom
    anom_lst.append(anom_sel)


anom_cb = xr.concat(anom_lst, dim='time') # Concatenate the anomalies along the time dimension

# Create a new dataset for anomalies
## case1 
anom_ds = xr.Dataset(
    data_vars = dict(
        new_anom = anom_cb
        ),
    coords = dict(
        lon = lon,
        lat = lat,
        time = time
    )
)

## case2
# anom_ds = xr.Dataset({'new_anom': anom_cb},
#                           coords={'time': new_clim.time, 'lat': new_clim.lat, 'lon': new_clim.lon})


c:\Users\ocpc\.conda\envs\cartopy\lib\site-packages\xarray\core\concat.py:500: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))


In [12]:
anom_ds

<xarray.Dataset>
Dimensions:   (time: 360, lon: 144, lat: 72)
Coordinates:
  * time      (time) datetime64[ns] 1991-01-01 1991-02-01 ... 2020-12-01
  * lon       (lon) float32 1.25 3.75 6.25 8.75 ... 351.2 353.8 356.2 358.8
  * lat       (lat) float32 88.75 86.25 83.75 81.25 ... -83.75 -86.25 -88.75
    month     (time) int32 1 2 3 4 5 6 7 8 9 10 11 ... 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    new_anom  (time, lat, lon) float32 8.318 7.295 7.43 ... -13.07 -13.2 -13.6

In [13]:
## apply attributes
anom_ds.lon.attrs = new_clim.lon.attrs # anom_ds.lon.attrs = {'units':'degree_east','long_name':'Longitude'}
anom_ds.lat.attrs = {'units':'degree_north','long_name': 'Latitude'}
anom_ds.new_anom.attrs ={'units':'mm/month','long_name':'anomaly Monthly Rate of Precipitation'}

In [14]:
os.getcwd()
# anom_ds.to_netcdf('8. CMAP_clim_anom_ncfile/individual_ncfile/CMAP_anom_new.nc')

### ◆ std nc file

In [16]:
## std
std_sel=new_clim.precip.groupby('time.month').std(dim='time',ddof=1)

## Change the moth dim to the time dim  len(month)->len(tim)e
std_extend_lst = []

for i in range(30):
    for j in std_sel.values:
            std_extend_lst.append(j)

std_extend_arr = np.array(std_extend_lst)
        
std_ds = xr.Dataset(
    data_vars=dict(
        new_std = (['time','lat','lon'],std_extend_arr)
    ),
    coords= dict(
        lon = lon,
        lat = lat,
        time = time
    )
)       

In [17]:
## apply attributes
std_ds.lon.attrs = {'units':'degree_east','long_name':'Longitude'}
std_ds.lat.attrs = {'units':'degree_north','long_name': 'Latitude'}
std_ds.new_std.attrs ={'unit':'mm/month','long_name':'Standard Deviation Rate of Precipitation'}

In [18]:
std_ds

<xarray.Dataset>
Dimensions:  (time: 360, lat: 72, lon: 144)
Coordinates:
  * lon      (lon) float32 1.25 3.75 6.25 8.75 11.25 ... 351.2 353.8 356.2 358.8
  * lat      (lat) float32 88.75 86.25 83.75 81.25 ... -83.75 -86.25 -88.75
  * time     (time) datetime64[ns] 1991-01-01 1991-02-01 ... 2020-12-01
Data variables:
    new_std  (time, lat, lon) float32 4.339 5.462 5.373 ... 14.17 14.36 14.89

In [19]:
os.getcwd()
# std_ds.to_netcdf('8. CMAP_clim_anom_ncfile/individual_ncfile/CMAP_std_new.nc')

# cut old clim

- precip(monthly mean)
- clim_ds
- anom_ds
- stdd_ds

In [3]:
# old_clim_ds
old_clim = ds_precip.sel(time=slice('1981-01-01','2010-12-01'))
old_clim

<xarray.Dataset>
Dimensions:  (time: 360, lat: 72, lon: 144)
Coordinates:
  * time     (time) datetime64[ns] 1981-01-01 1981-02-01 ... 2010-12-01
  * lon      (lon) float32 1.25 3.75 6.25 8.75 11.25 ... 351.2 353.8 356.2 358.8
  * lat      (lat) float32 88.75 86.25 83.75 81.25 ... -83.75 -86.25 -88.75
Data variables:
    precip   (time, lat, lon) float32 ...

In [4]:
# Unit : mm/day -> mm/month
for i in range(len(old_clim.precip)):
    p_time = str(old_clim.time[i].values)[:10] # '1979-03-01'
    b = datetime.datetime.strptime(p_time,'%Y-%m-%d')
    nums_of_day = ((b + relativedelta(months=i))-datetime.timedelta(days=1)).day
    old_clim.precip[i] *= nums_of_day
precip = old_clim.precip

lon = old_clim.lon
lat = old_clim.lat
time = old_clim.time

## creat clim sel
clim_sel = old_clim['precip'].groupby('time.month').mean('time')
old_clim['old_clim'] = clim_sel


### ◆ precip nc file (monthly mean)

In [18]:
## monthly ds
precip_arr = np.array(precip)

precip_ds = xr.Dataset(
    data_vars= dict(
        precip = (['time','lat','lon'], precip_arr)
    ),
    coords=dict(
        lon = lon,
        lat = lat,
        time = time
    )
)

## apply attributes
precip_ds.lon.attrs = {'units':'degree_east','long_name':'Longitude'}
precip_ds.lat.attrs = {'units':'degree_north','long_name': 'Latitude'}
precip_ds.precip.attrs ={'units':'mm/month','long_name':'Average Monthly Rate of Precipitation'}


In [19]:
precip_ds

<xarray.Dataset>
Dimensions:  (time: 360, lat: 72, lon: 144)
Coordinates:
  * lon      (lon) float32 1.25 3.75 6.25 8.75 11.25 ... 351.2 353.8 356.2 358.8
  * lat      (lat) float32 88.75 86.25 83.75 81.25 ... -83.75 -86.25 -88.75
  * time     (time) datetime64[ns] 1981-01-01 1981-02-01 ... 2010-12-01
Data variables:
    precip   (time, lat, lon) float32 13.64 13.64 13.33 13.02 ... 5.89 5.89 5.89

### ◆ clim nc file

In [31]:
clim_sel = old_clim['precip'].groupby('time.month').mean('time')

## Change the moth dim to the time dim # month-> time
clim_extend_lst = []

for i in range(30):
    for j in clim_sel.values:
            clim_extend_lst.append(j)

clim_extend_arr = np.array(clim_extend_lst)
        
clim_ds = xr.Dataset(
    data_vars=dict(
        old_clim = (['time','lat','lon'],clim_extend_arr)
    ),
    coords= dict(
        lon = lon,
        lat = lat,
        time = time
    )
)     

## apply attributes
clim_ds.lon.attrs = {'units':'degree_east','long_name':'Longitude'}
clim_ds.lat.attrs = {'units':'degree_north','long_name': 'Latitude'}
clim_ds.old_clim.attrs ={'units':'mm/month','long_name':'Average Long Term Mean Monthly Rate of Precipitation'}  

In [32]:
# os.getcwd()
# clim_ds.to_netcdf('8. CMAP_clim_anom_ncfile/individual_ncfile/CMAP_clim_old.nc')

### ◆ anom nc file

In [27]:
anom_lst = []
for  q in range(1, 13):
    mon_group = old_clim.sel(time=old_clim['time.month'] == q)  # month
    anom_sel = mon_group['precip'] - clim_sel.sel(month=q)  # anom
    anom_lst.append(anom_sel)


anom_cb = xr.concat(anom_lst, dim='time') # Concatenate the anomalies along the time dimension

# Create a old dataset for anomalies
## case1 
anom_ds = xr.Dataset(
    data_vars = dict(
        old_anom = anom_cb
        ),
    coords = dict(
        lon = lon,
        lat = lat,
        time = time
    )
)

## case2
# anom_ds = xr.Dataset({'old_anom': anom_cb},
#                           coords={'time': old_clim.time, 'lat': old_clim.lat, 'lon': old_clim.lon})


c:\Users\ocpc\.conda\envs\cartopy\lib\site-packages\xarray\core\concat.py:500: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))


In [28]:
## apply attributes
anom_ds.lon.attrs = new_clim.lon.attrs
anom_ds.lat.attrs = {'units':'degree_north','long_name': 'Latitude'}
anom_ds.old_anom.attrs ={'units':'mm/month','long_name':'anomaly Monthly Rate of Precipitation'}

In [29]:
os.getcwd()
# anom_ds.to_netcdf('8. CMAP_clim_anom_ncfile/individual_ncfile/CMAP_anom_old.nc')

### ◆ std nc file

In [5]:
## std
std_sel=old_clim.precip.groupby('time.month').std(dim='time',ddof=1)

## Change the moth dim to the time dim
std_extend_lst = []

for i in range(30):
    for j in std_sel.values:
            std_extend_lst.append(j)

std_extend_arr = np.array(std_extend_lst)
        
std_ds = xr.Dataset(
    data_vars=dict(
        old_std = (['time','lat','lon'],std_extend_arr)
    ),
    coords= dict(
        lon = lon,
        lat = lat,
        time = time
    )
)       

## apply attributes
std_ds.lon.attrs = {'units':'degree_east','long_name':'Longitude'}
std_ds.lat.attrs = {'units':'degree_north','long_name': 'Latitude'}
std_ds.old_std.attrs ={'unit':'mm/month','long_name':'Standard Deviation Rate of Precipitation'}

In [7]:
os.getcwd()
# std_ds.to_netcdf('8. CMAP_ncfile(mean,clim,anom,std)/individual_ncfile/CMAP_std_old.nc')